In [1]:
!pip install torchcrf
!pip install tokenizers
!pip install huggingface_hub
!pip install transformers
!pip install sentencepiece
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 10.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 163 kB 9.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 14.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 15.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 880 kB 16.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=061f420129ae53bb5e07c0

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Import modules

In [3]:
from torch.utils.data import Dataset, DataLoader
from TorchCRF import CRF
from tokenizers import pre_tokenizers
from tokenizers.pre_tokenizers import Whitespace, Punctuation

import torch
import pandas as pd
import torch.nn as nn
import json
import math
import logging
import tqdm

In [4]:
import sys    
path_to_module = '/content/gdrive/MyDrive/IT458_project'
sys.path.append(path_to_module)

from utils import TRANSFORMER_PATH, LABEL_MAPPING, convert_examples_to_features, get_data


# Dataset preprocessing

In [5]:
path = "/content/gdrive/MyDrive/IT458_project/"

In [6]:
class GunViolenceDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        return self.texts[index], self.labels[index]

In [7]:
def _gen_label(words, target):
    target_len = len(target)
    tags = ['O'] * len(words)
    for i in range(0, len(words)):
        try:
            if ' '.join(words[i:i+target_len]) == ' '.join(target):
                tags[i] = 'B'
                for j in range(i+1, i+target_len):
                    tags[j] = 'I'
        except IndexError as e:
            print(e)
            exit()

    return ' '.join(tags)

def preprocess(input_file, output_file, target_type):
    df = pd.read_csv(input_file, sep='\t')
    texts = df['Full text'].tolist()
    jsons = df['Json'].tolist()

    new_texts = []
    labels = []

    for text, data in zip(texts, jsons):
        try:
            # use BERT tokenizer to process whitespace and punctuaction
            data = json.loads(data)
            text = text.replace('\u200b', '')
            pre_tokenizer = pre_tokenizers.Sequence([Whitespace(), Punctuation()])
            tokenized_text = [t[0] for t in pre_tokenizer.pre_tokenize_str(text)]
            target = data[target_type + '-section'][0]['name']['value']
            tokenized_target = [t[0] for t in pre_tokenizer.pre_tokenize_str(target)]

            # if no target or empty in array, mark every token as O
            if not target:
                raise IndexError

            # generate labels for each tokenized token
            label = _gen_label(tokenized_text, tokenized_target)

            # keep tokenized text that has less than 512 text-length
            if label and len(tokenized_text) < 512:
                new_texts.append(' '.join(tokenized_text))
                labels.append(label)

        except IndexError:
            # mark every token as O
            new_texts.append(' '.join(tokenized_text))
            label = ' '.join(['O'] * len(tokenized_text))
            labels.append(label)

    data = list(zip(new_texts, labels))
    df = pd.DataFrame(data)
    df.columns = ['texts', 'labels']
    df.to_csv(output_file, index=False)

In [8]:
target = "victim"

preprocess(path + 'dataset/train.tsv', path + target + '/train.csv', target)
preprocess(path + 'dataset/dev.tsv', path + target + '/dev.csv', target)
preprocess(path + 'dataset/test.tsv', path + target + '/test.csv', target)

In [9]:
target = "shooter"

preprocess(path + 'dataset/train.tsv', path + target + '/train.csv', target)
preprocess(path + 'dataset/dev.tsv', path + target + '/dev.csv', target)
preprocess(path + 'dataset/test.tsv', path + target + '/test.csv', target)

# Models

### BERT_Linear

In [10]:
class BERT_Linear(nn.Module):
    def __init__(self, num_labels):
        super(BERT_Linear, self).__init__()
        config = torch.hub.load(TRANSFORMER_PATH, 'config', 'bert-base-cased')
        config.max_position_embeddings = 1024
        self.bert = torch.hub.load(TRANSFORMER_PATH, 'model', 'bert-base-cased')
        self.classifier = nn.Linear(768, num_labels)

    def forward(self, tokens_tensor, segments_tensors, labels=None):
        bert_output = self.bert(tokens_tensor, token_type_ids=segments_tensors)
        last_hidden_state = bert_output.last_hidden_state
        pooler_output = bert_output.pooler_output

        logits = self.classifier(last_hidden_state)
        return logits

### BERT_LSTM

In [11]:
class BERT_LSTM(nn.Module):
    def __init__(self, num_labels):
        super(BERT_LSTM, self).__init__()
        config = torch.hub.load(TRANSFORMER_PATH, 'config', 'bert-base-cased')
        config.max_position_embeddings = 1024
        self.bert = torch.hub.load(TRANSFORMER_PATH, 'model', 'bert-base-cased')
        self.lstm = nn.LSTM(768, 768)
        self.classifier = nn.Linear(768, num_labels)

    def forward(self, tokens_tensor, segments_tensors, labels=None):
        bert_output = self.bert(tokens_tensor, token_type_ids=segments_tensors)
        last_hidden_state = bert_output.last_hidden_state
        pooler_output = bert_output.pooler_output

        lstm_out, _ = self.lstm(last_hidden_state)
        logits = self.classifier(lstm_out)
        return logits

### BERT_BiLSTM

In [12]:
class BERT_BiLSTM(nn.Module):
    def __init__(self, num_labels):
        super(BERT_BiLSTM, self).__init__()
        config = torch.hub.load(TRANSFORMER_PATH, 'config', 'bert-base-cased')
        config.max_position_embeddings = 1024
        self.bert = torch.hub.load(TRANSFORMER_PATH, 'model', 'bert-base-cased')
        self.lstm = nn.LSTM(768, 768, bidirectional=True)
        self.classifier = nn.Linear(768, num_labels)
        # self.classifier = nn.Linear(768 * 2, num_labels)

    def forward(self, tokens_tensor, segments_tensors, labels=None):
        bert_output = self.bert(tokens_tensor, token_type_ids=segments_tensors)
        last_hidden_state = bert_output.last_hidden_state
        pooler_output = bert_output.pooler_output

        lstm_out, _ = self.lstm(last_hidden_state)
        lstm_out = lstm_out[:, :, :768] + lstm_out[:, :, 768:]
        logits = self.classifier(lstm_out)
        return logits

# Train and Evaluate model

In [14]:
def evaluate(model, evaluate_X, evaluate_Y, tokenizer, cuda_available, batch_size, max_seq_length, model_type, lr, epochs, output_dir):

    def _get_prediction(normalized_probs):
        # classify B, I, O based on probabilities
        labels = []
        for sample_prob in normalized_probs:
            max_prob = -math.inf
            label = None
            for i, prob in enumerate(sample_prob):
                if max_prob < prob:
                    max_prob = prob
                    label = i
            labels.append(label)
        return labels

    model.eval()
    num_samples = len(evaluate_X)
    evaluate_set = GunViolenceDataset(evaluate_X, evaluate_Y)
    evaluate_generator = DataLoader(
        evaluate_set,
        batch_size=1,
        shuffle=True,
    )
    num_of_tp = num_of_fn = num_of_fp = num_of_tn = 0

    for i, (evaluate_x, evaluate_y) in enumerate(evaluate_generator):
        tokens, labels = convert_examples_to_features(evaluate_x, evaluate_y, tokenizer, max_seq_length)

        indexed_tokens = [tokenizer.convert_tokens_to_ids(token) for token in tokens]
        segments_ids = [[0] * len(indexed_token) for indexed_token in indexed_tokens]

        if cuda_available:
            segments_tensors = torch.tensor(segments_ids).to('cuda')
            tokens_tensor = torch.tensor(indexed_tokens).to('cuda')
            labels = torch.tensor(labels).to('cuda')
        else:
            segments_tensors = torch.tensor(segments_ids)
            tokens_tensor = torch.tensor(indexed_tokens)
            labels = torch.tensor(labels)

        with torch.no_grad():
            y_pred = model(tokens_tensor, segments_tensors, labels)
            y_pred = y_pred[0]  # because batch size is 1, we just take the 1st row
            normalized_probs = nn.functional.softmax(y_pred, dim=1)
            results = _get_prediction(normalized_probs)

            # get the real target
            original = ''
            for i, (x, y) in enumerate(zip(evaluate_x[0].split(), evaluate_y[0].split())):
                if y[0] == 'B':
                    original = x + ' '
                    index = i
                    while index + 1 < len(evaluate_y[0].split()) and evaluate_y[0].split()[index + 1][0] == 'I':
                        original += '{} '.format(evaluate_x[0].split()[index + 1])
                        index += 1
                    break
            original = original.strip()

            probabilities = []
            predictions = []
            prediction = []
            for token, tag, prob in zip(tokens[0], results, normalized_probs):
                if tag == 0:
                    # tag == 'B'
                    probabilities.append(prob)

                    if len(prediction) != 0:
                        predictions.append(prediction)
                        prediction = []
                    prediction.append(token)
                elif tag == 1:
                    # tag == 'I'
                    prediction.append(token)
            if len(prediction) != 0:
                predictions.append(prediction)

            # one sentence might generate multiple targets, eg. shooters or victims
            # we need to pick the most possible one, which is the one has the highest probability in 'B' tag
            max_prob = -math.inf
            max_prob_ind = 0
            for i, prob in enumerate(probabilities):
                if max_prob < prob[0]:
                    max_prob_ind = i
                    max_prob = prob[0]

            # calculate true positive, false positive, true negative, false negative
            result = ''
            if len(predictions) != 0:
                result = tokenizer.convert_tokens_to_string(predictions[max_prob_ind])
                if result == original:
                    num_of_tp += 1
                else:
                    num_of_fp += 1
            else:
                if original.strip() != '':
                    num_of_fn += 1
                else:
                    num_of_tn += 1

    accuracy = (num_of_tp + num_of_tn) /num_samples if num_samples != 0 else 0
    precision = num_of_tp/(num_of_tp + num_of_fp) if num_of_tp + num_of_fp != 0 else 0
    recall = num_of_tp/(num_of_tp + num_of_fn) if num_of_tp + num_of_fn != 0 else 0

    if(get_accuracy):
      print('Accuracy : {}'.format(accuracy))
    else:
      print("\nEvaluation :\n")
      print('True positive : {}'.format(num_of_tp))
      print('False positive : {}'.format(num_of_tp))
      print('True negative : {}'.format(num_of_tp))
      print('False negative : {}'.format(num_of_tp))
      print('Accuracy : {}'.format(accuracy))
      print('Precision : {}'.format(precision))
      print('Recall : {}'.format(recall))
      print('F1_score : {}'.format(2 * precision * recall / (precision + recall) if precision + recall != 0 else 0))

In [13]:
def train(train_X, train_Y, learning_rate, cuda_available, epochs, model_type, is_balance, batch_size, max_seq_length, patience, min_delta, baseline, path, evaluate_X, evaluate_Y, lr):

    # Set up DataLoader for populating batches later
    training_set = GunViolenceDataset(train_X, train_Y)
    training_generator = DataLoader(
        training_set,
        batch_size=batch_size,
        shuffle=True,
    )
    iter_in_one_epoch = len(train_X) // batch_size

    # Set up tokenizer and model
    tokenizer = torch.hub.load(TRANSFORMER_PATH, 'tokenizer', 'bert-base-cased') # cased!
    model = None
    if model_type == 'LSTM':
        model = BERT_LSTM(3)
    elif model_type == 'BiLSTM':
        model = BERT_BiLSTM(3)
    else:
        model = BERT_Linear(3)  # 3 different labels: B, I, O
    if cuda_available:
        model.to('cuda')  # move data onto GPU
    model.train()

    # Set up variables
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    losses = []
    num_no_improve = 0
    best_loss = None
    stopping_epoch = 0

    print("\nTraining :\n")

    for epoch in range(1, epochs + 1):
        loss = 0
        with tqdm.tqdm(training_generator, unit="batch") as tepoch:
            for i, (train_x, train_y) in enumerate(tepoch):
                tepoch.set_description("Epoch {}".format(epoch))

                # prepare model input
                tokens, labels = convert_examples_to_features(train_x, train_y, tokenizer, max_seq_length)
                indexed_tokens = [tokenizer.convert_tokens_to_ids(token) for token in tokens]
                segments_ids = [[0] * len(indexed_token) for indexed_token in indexed_tokens]

                if cuda_available:
                    segments_tensors = torch.tensor(segments_ids).to('cuda')
                    tokens_tensor = torch.tensor(indexed_tokens).to('cuda')
                    labels = torch.tensor(labels).to('cuda')
                else:
                    segments_tensors = torch.tensor(segments_ids)
                    tokens_tensor = torch.tensor(indexed_tokens)
                    labels = torch.tensor(labels)

                # forward pass
                y_pred = model(tokens_tensor, segments_tensors, labels)
                y_pred = y_pred.permute(0, 2, 1)

                # calculate loss
                loss_fct = nn.CrossEntropyLoss()
                loss = loss_fct(y_pred, labels)
                losses.append((epoch + i / iter_in_one_epoch, loss.item()))

                # display loss
                tepoch.set_postfix(loss="{:.4f}".format(loss.item()))

                # zero out gradients
                optimizer.zero_grad()

                # backward pass
                loss.backward()

                # update parameters
                optimizer.step()
                
            # early stop
            if not best_loss:
                # first iteration
                best_loss = loss
            elif loss <= best_loss + min_delta:
                best_loss = loss
                num_no_improve += 1
            elif loss < baseline:
                num_no_improve += 1
            if num_no_improve > patience:
                stopping_epoch = epoch
                logging.info('Early Stop on epoch {} with the best loss {}'.format(stopping_epoch, best_loss))
                break

        torch.save(model, path + model_type + "_model")
        evaluate(model, evaluate_X, evaluate_Y, tokenizer, cuda_available, batch_size, max_seq_length, model_type, lr, epochs, path, True)

    return model, tokenizer, stopping_epoch

In [15]:
lr = 1e-4
epochs = 7
batch_size = 40
max_seq_length = 256
is_balance = True
patience = 10
min_delta = 0
baseline = 0.0001
cuda_available = torch.cuda.is_available()

### For victim

In [16]:
input_dir = path + "victim"
output_dir = path + "victim/"
train_X, train_Y = get_data(input_dir + '/train.csv', is_balance)
dev_X, dev_Y = get_data(input_dir + '/dev.csv', is_balance)
test_X, test_Y = get_data(input_dir + '/test.csv')
train_X += dev_X
train_Y += dev_Y

In [ ]:
model, tokenizer, stopping_epoch = train(train_X, train_Y, lr, cuda_available, epochs, "Linear", is_balance, batch_size, max_seq_length, patience, min_delta, baseline, output_dir, test_X, test_Y, lr )
evaluate(model, test_X, test_Y, tokenizer, cuda_available, batch_size, max_seq_length, "Linear", lr, stopping_epoch, output_dir)

/usr/local/lib/python3.7/dist-packages/torch/hub.py:267: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  "You are about to download and run code from an untrusted repository. In a future release, this won't "
Downloading: "https://github.com/huggingface/pytorch-transformers/zipball/main" to /root/.cache/torch/hub/main.zip


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main
Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Training :



Epoch 1:   1%|          | 5/789 [00:09<22:29,  1.72s/batch, loss=0.1293]

In [ ]:
model, tokenizer, stopping_epoch = train(train_X, train_Y, lr, cuda_available, epochs, "LSTM", is_balance, batch_size, max_seq_length, patience, min_delta, baseline, output_dir, test_X, test_Y, lr  )
evaluate(model, test_X, test_Y, tokenizer, cuda_available, batch_size, max_seq_length, "LSTM", lr, stopping_epoch, output_dir)

In [ ]:
model, tokenizer, stopping_epoch = train(train_X, train_Y, lr, cuda_available, epochs, "BiLSTM", is_balance, batch_size, max_seq_length, patience, min_delta, baseline, output_dir, test_X, test_Y, lr )
evaluate(model, test_X, test_Y, tokenizer, cuda_available, batch_size, max_seq_length, "BiLSTM", lr, stopping_epoch, output_dir)

### For shooter

In [ ]:
input_dir = path + "shooter"
output_dir = path + "shooter/output"
train_X, train_Y = get_data(input_dir + '/train.csv', is_balance)
dev_X, dev_Y = get_data(input_dir + '/dev.csv', is_balance)
test_X, test_Y = get_data(input_dir + '/test.csv')
train_X += dev_X
train_Y += dev_Y

In [ ]:
model, tokenizer, stopping_epoch = train(train_X, train_Y, lr, cuda_available, epochs, "Linear", is_balance, batch_size, max_seq_length, patience, min_delta, baseline, output_dir, test_X, test_Y, lr  )
evaluate(model, test_X, test_Y, tokenizer, cuda_available, batch_size, max_seq_length, "Linear", lr, stopping_epoch, output_dir)

In [ ]:
model, tokenizer, stopping_epoch = train(train_X, train_Y, lr, cuda_available, epochs, "LSTM", is_balance, batch_size, max_seq_length, patience, min_delta, baseline, output_dir, test_X, test_Y, lr  )
evaluate(model, test_X, test_Y, tokenizer, cuda_available, batch_size, max_seq_length, "LSTM", lr, stopping_epoch, output_dir)

In [ ]:
model, tokenizer, stopping_epoch = train(train_X, train_Y, lr, cuda_available, epochs, "BiLSTM", is_balance, batch_size, max_seq_length, patience, min_delta, baseline, output_dir, test_X, test_Y, lr  )
evaluate(model, test_X, test_Y, tokenizer, cuda_available, batch_size, max_seq_length, "BiLSTM", lr, stopping_epoch, output_dir)